In [22]:
train_path = 'data/training.csv'
test_path = 'data/test.csv'
look_up_path = 'data/IdLookupTable.csv'

train_data = pd.read_csv(train_path)

In [16]:
import pandas as pd
from PIL import Image
import numpy as np

In [23]:
def print_image(image_ID):
    image_raw = []
    formatted_image = []
    for num in train_data['Image'][image_ID].split(' '):
        image_raw.append(int(num))

    for i in range(96):
        row = []
        for j in range(96):
            row.append(image_raw[i*96+j])
        formatted_image.append(row)

    img = Image.fromarray(np.array(formatted_image, dtype=np.uint8), mode='L')
    img.show()

print_image(0)